In [11]:
import numpy as np
import random, json
import nltk_utils

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from nltk_utils import bag_words, tokeniser, stem
import model
from model import Network

In [12]:
with open('intents.json','r') as f:
    intents=json.load(f)

Words=[]
tags=[]
xy=[]

In [13]:
for intent in intents['intents']:
    tag=intent['tag']
    tags.append(tag)
    
    for pattern in intent['patterns']:
        w=tokeniser(pattern)
        Words.extend(w)
        xy.append((w,tag))
words_to_ignore=['?',".",'!']
Words=[stem(w) for w in Words if w not in words_to_ignore]

In [14]:
#Sorting all words, removing duplicated
Words=sorted(set(Words))
tags= sorted(set(tags))

print(len(xy), "patterns")

print(len(tags), "tags:", tags)
print(len(Words), "Unique words", Words)


26 patterns
7 tags: ['delivery', 'funny', 'goodbye', 'greeting', 'items', 'payments', 'thanks']
54 Unique words ["'s", 'a', 'accept', 'anyon', 'are', 'bye', 'can', 'card', 'cash', 'credit', 'day', 'deliveri', 'do', 'doe', 'funni', 'get', 'good', 'goodby', 'have', 'hello', 'help', 'hey', 'hi', 'how', 'i', 'is', 'item', 'joke', 'kind', 'know', 'later', 'long', 'lot', 'mastercard', 'me', 'my', 'of', 'onli', 'pay', 'paypal', 'see', 'sell', 'ship', 'someth', 'take', 'tell', 'thank', 'that', 'there', 'what', 'when', 'which', 'with', 'you']


In [15]:
#Creating trainin dataset

X_train=[]
Y_train=[]


for (pattern_sentence, tag) in xy:
    #X: Bag of words for each pattern sentence
    bag= bag_words(pattern_sentence, Words)
    X_train.append(bag)
    #Y: Class labels
    
    label=tags.index(tag)
    Y_train.append(label)
    
X_train=np.array(X_train)
Y_train=np.array(Y_train)









In [16]:
#Specifying hyper parameters

eporchs=1000
batch_size=8
lr=0.001
inputs=len(X_train[0])
hidden=8
outputs= len(tags)

print(inputs, outputs)



54 7


In [17]:
#creating chatdat class to load dataset
class ChataData(Dataset):
    def __init__(self):
        self.n_samples= len(X_train)
        
        self.x_data= X_train
        self.y_data= Y_train
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.n_samples


In [18]:
chat_data=ChataData()
#loading the trainloader
train_loader= DataLoader(dataset=chat_data, batch_size=batch_size, shuffle=True, num_workers=0)

device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model= Network(inputs, hidden, outputs).to(device)
criterion=nn.CrossEntropyLoss()
optimiser=torch.optim.Adam(model.parameters(), lr=lr)


In [19]:
#training the neural network
for epoch in range(eporchs):
    for(words, labels) in train_loader:
        words= words.to(device)
       
        labels=labels.to(dtype=torch.long).to(device)
        
        #forward pass
        outputs= model(words)
        
        loss=criterion(outputs, labels)
        
        #backprop and optimisation
        
        optimiser.zero_grad()
        
        loss.backward()
        optimiser.step()
        
        if(epoch+1)%100==0:
            print(f'Epoch [{epoch+1}/{eporchs}], Loss: {loss.item():.4f}')
print(f'final loss: {loss.item():.4f}')


Epoch [100/1000], Loss: 1.1471
Epoch [100/1000], Loss: 1.1813
Epoch [100/1000], Loss: 0.9249
Epoch [100/1000], Loss: 1.0999
Epoch [200/1000], Loss: 0.1015
Epoch [200/1000], Loss: 0.0917
Epoch [200/1000], Loss: 0.1836
Epoch [200/1000], Loss: 0.2116
Epoch [300/1000], Loss: 0.0161
Epoch [300/1000], Loss: 0.0241
Epoch [300/1000], Loss: 0.0333
Epoch [300/1000], Loss: 0.0085
Epoch [400/1000], Loss: 0.0071
Epoch [400/1000], Loss: 0.0094
Epoch [400/1000], Loss: 0.0070
Epoch [400/1000], Loss: 0.0204
Epoch [500/1000], Loss: 0.0031
Epoch [500/1000], Loss: 0.0047
Epoch [500/1000], Loss: 0.0060
Epoch [500/1000], Loss: 0.0031
Epoch [600/1000], Loss: 0.0031
Epoch [600/1000], Loss: 0.0018
Epoch [600/1000], Loss: 0.0026
Epoch [600/1000], Loss: 0.0042
Epoch [700/1000], Loss: 0.0019
Epoch [700/1000], Loss: 0.0012
Epoch [700/1000], Loss: 0.0021
Epoch [700/1000], Loss: 0.0011
Epoch [800/1000], Loss: 0.0007
Epoch [800/1000], Loss: 0.0014
Epoch [800/1000], Loss: 0.0011
Epoch [800/1000], Loss: 0.0022
Epoch [9

In [20]:
#savind the trained model
models={
    "model_state": model.state_dict(),
    "inputs": inputs,
    "hidden": hidden,
    "outputs": outputs,
    "tags": tags,
    "Words": Words
}

File="models.pth"

torch.save(models, File)

print(f'training complete. Model saved to {File}')

training complete. Model saved to models.pth
